In [1]:
import pandas as pd
import geopandas as gpd
import h3pandas
import numpy as np
import shapely
import json


In [2]:
"""
This cell is only for cleaning data, 
leave commented out unless you don't already have the clean data.
"""

# # load raw data from file
# cblocks = gpd.read_file('../data/raw/census_blocks.csv')

# # renames first column to correct geopandas format
# cblocks.columns = ['geometry', 'CB2020', 'BoroCode', 'BoroName', 'CT2020', 'BCTCB2020',
#        'GEOID', 'Shape_Leng', 'Shape_Area']
    
# # dictionary of each borocode
# boro_dict = {2:'bronx', 3:'brooklyn', 1:'manhattan', 4:'queens', 5:'staten'}

# # writes each borough to its own file
# for boro_code in range(1,6):
#     cblocks[cblocks["BoroCode"]== str(boro_code)].to_csv(f"../data/clean/census_blocks/{boro_dict[boro_code]}.csv",index=False)
    
# df = pd.read_csv('../data/raw/pluto_22v1.csv',low_memory =False)

gdf_in = gpd.read_file("/home/thowe/Downloads/nyc_mappluto_22v1_shp/MapPLUTO.shp")


In [3]:
initial_resolution = 8
parent_resolution = 10

In [4]:
# len(gdf['BBL']) - len(set(gdf['BBL'])) 
# len(set(df['bbl']))
gdf = gdf_in
# gdf['Latitude'].isna().sum()

In [6]:
gdf = gdf.dropna(subset='BCTCB2020')
gdf['BCTCB2020'] = gdf['BCTCB2020'].astype(int)
gdf['BBL'] = gdf['BBL'].astype(int)

In [7]:
h3_df = gdf[['BBL','BCTCB2020','Latitude','Longitude']]
h3_df = h3_df.rename({'Latitude':'lat','Longitude':'lon'},axis=1)

h3_df

,BBL,BCTCB2020,lat,lon
2,1000020002,10009001022,40.700550,-74.011588
3,1000030001,10013002002,40.702806,-74.015631
4,1000030010,10319001006,40.700918,-74.014444
5,1000050010,10009001014,40.702776,-74.010722
6,1000070027,10009001015,40.703289,-74.011022
...,...,...,...,...
857001,5080500037,50248001014,40.507801,-74.251802
857002,5080500062,50248001014,40.508640,-74.250938
857003,5080500065,50248001014,40.508783,-74.250856
857004,5080500092,50248001014,40.508794,-74.249856


In [9]:
h3_df = h3_df.h3.geo_to_h3(parent_resolution,lat_col='lat',lng_col='lon')
out_df = h3_df.reset_index().set_index('BBL')[['h3_10']]
len(set(out_df.index)) - len(out_df.index)
out_df.to_json('../data/clean/bbl_to_h3.json', orient='index')

In [12]:
out = gdf_in.merge(out_df,on='BBL')

In [14]:
out.to_csv('../data/clean/h3_PLUTO.csv')